In [1]:
from notebook.services.config import ConfigManager
cfgm = ConfigManager()
cfgm.update('livereveal', {
              'theme': 'simple',
              'transition': 'convex',
              'start_slideshow_at': 'selected'
});

In [2]:
from IPython.core.display import HTML
HTML("""<style>
.answers { 
    visibility: hidden;
}
</style>""")

# Python Emissions Perturbation
Author: Barron H. Henderson

# CMAQ: Emissions Perturbation

1. Copy a standard emissions input file
2. Open the file in edit mode
3. Change the concentration (e.g., cut NOx by half)
4. Close the file

In [ ]:
emisoldpath = 'emis/'
emisnewpath = 'new_emis.nc'

# CMAQ: Copy a standard emissions file

In [3]:
import shutil
shutil.copyfile(emisoldpath, emisnewpath)

# CMAQ: Open the new file in edit mode

In [ ]:
emisargs = PNC('--format=netcdf,mode="r+"', emisnewpath)
emisnewfile = emisargs.ifiles[0]
emisnewfile.variables['NO'][:] /= 2
emisnewfile.variables['NO2'][:] /= 2
emisnewfile.close()

# CMAQ: reopen and verify

In [4]:
emisargs = PNC("--format=netcdf", emisoldpath, emisnewpath)
oldfile, newfile = emisargs.ifiles
print('SPC, OLD, NEW, UNITS')
Mw = dict(NO = 30, NO2 = 46) # g/mole
t2h = 3600
g2Mg = 1e-6
for k in 'NO NO2'.split():
    old = oldfile.variables[k][:].sum()*t2h*g2Mg*Mw[k]
    new = newfile.variables[k][:].sum()*t2h*g2Mg*Mw[k]
    print('%3s %5.1f %5.1f Mg/day' % (k, old, new))

NameError: name 'emisfile' is not defined

# CAMx: Emissions Perturbation

Repeat the CMAQ process with two exceptions:
1. use uamiv instead of netcdf
2. remove the change t2h from 3600 (what should it be?)